# Compilateur d'exercices de séparation de mots
- problème
 - les scores non-nuls ne peuvent pas utiliser les négations ?

In [16]:
#import re
import sys, codecs, optparse, glob, os
import string
import random

In [17]:
debug=True
dossier="/Github/eGloss"
enonce="Test"
theme="SeparationMots"
adaptatifPenalite="0.1000000"
dernierIndiceNote=75.0
msgPlusDeMots=u"Il y a plus de mots"
msgMoinsDeMots=u"Il y a moins de mots"

nbIndiceNotes={
    1:["%50%"],
    2:["","%50%"],
    3:["","%40%","%75%"],
    4:["","%25%","%50%","%75%"],
    5:["","%25%","%40%", "%60%","%75%"],
    6:["","%20%","%33.33333%","%50%", "%60%","%75%"],
    7:["","%16.66667%","%25%","%33.33333%","%50%", "%60%","%75%"],
    8:["","%16.66667%","%25%","%33.33333%","%45%","%55%", "%66.66667%","%75%"],
    9:["","%5%","%16.66667%","%25%","%33.33333%","%45%","%55%", "%66.66667%","%75%"],
    10:["","%5%","%16.66667%","%25%","%33.33333%","%40%","%50%", "%60%","%66.66667%","%75%"],
}

In [18]:
dictIndiceNotes={
    1:["50"],
    2:["0","50"],
    3:["0","40","75"],
    4:["0","25","50","75"],
    5:["0","25","40", "60","75"],
    6:["0","20","33.33333","50", "60","75"],
    7:["0","16.66667","25","33.33333","50", "60","75"],
    8:["0","16.66667","25","33.33333","45","55", "66.66667","75"],
    9:["0","5","16.66667","25","33.33333","45","55", "66.66667","75"],
    10:["0","5","16.66667","25","33.33333","40","50", "60","66.66667","75"],
}

In [19]:
def nbIndiceNotes(nb,percent=1):
    return ["%"*percent+el+"%"*percent for el in dictIndiceNotes[nb]]

In [20]:
if debug:
    for i in range(10):
        print i+1,nbIndiceNotes(i+1,percent=0) 

1 ['50']
2 ['0', '50']
3 ['0', '40', '75']
4 ['0', '25', '50', '75']
5 ['0', '25', '40', '60', '75']
6 ['0', '20', '33.33333', '50', '60', '75']
7 ['0', '16.66667', '25', '33.33333', '50', '60', '75']
8 ['0', '16.66667', '25', '33.33333', '45', '55', '66.66667', '75']
9 ['0', '5', '16.66667', '25', '33.33333', '45', '55', '66.66667', '75']
10 ['0', '5', '16.66667', '25', '33.33333', '40', '50', '60', '66.66667', '75']


In [21]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [22]:
from ToolsMoodleXML2 import *
import yaml

In [23]:
yamlFichier=base+theme+"-"+enonce+".yaml"
csvFichier=base+theme+".txt"
boolEX=True
titreNoEX=False

In [24]:
with open(yamlFichier, 'r') as stream:
    configuration=yaml.load(stream)
    
with codecs.open(csvFichier, 'r', encoding="utf8") as file:
    donneesLignes=file.readlines()

In [25]:
def donneesEX():
    titre=[]
    result={}
    boolMultiLignes=False
    i=0
    for ligne in donneesLignes:
        if debug or 0: print ligne
        i+=1
        if ligne.startswith("#EX;"):
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#EXM;"):
            boolMultiLignes=True
            if result:
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                titre=temp[1:]            
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            if boolMultiLignes:
                result["boucle"].extend(temp)
            else:
                result["boucle"].append(temp)
#            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

donnees=[]
if boolEX or donneesLignes[0].startswith("#EX;"):
#    print "avec #EX;"
    donneesEX()
else:
#    print "sans #EX;"
    donneesNoEX()

In [26]:
def lierMot(ref,chaine):
    if motReferences[ref]:
        mot=motReferences[ref]
    else:
        mot=ref
    result=chaine.replace(ref,u'<a href="https://fr.wikipedia.org/wiki/%s" target="_blank" style="color:blue">%s</a>'%(mot,ref))
    return result

def echapperCorrection(correction):
    result=correction
    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\\[\1\\]",corps)
        result=prefixe+corps+suffixe
    print "échappé",result
    return result    

def modifierCorrection(correction):
    result=correction
#    print "original",result
    subPoints = re.compile(ur"(\.[^*+])")
    result=subPoints.sub(ur"\\\1",result)
    m=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",result)
    if m:
        (prefixe, corps, suffixe)=m.groups()        
        subParentheses = re.compile(ur"\(([^)]*)\)")
        corps=subParentheses.sub(ur"\\(\1\\)",corps)
        subCrochets = re.compile(ur"\[([^]]*)\]")
        corps=subCrochets.sub(ur"\\[\1\\]",corps)
        unites=filter(None,corps.split("&&"))
#        print "unités",unites
        corps=".*".join(unites)
        result=prefixe+corps+suffixe
#    print "échappé",result
    return result    

In [27]:
def separerMots(elements,titre):
    result=[]
    elements = filter(None, elements)
    nbMots=len(elements[0].strip().split(" "))
    solution=("100",elements[0],u"Bravo ! La phrase était : %s"%titre)
    corrections=[]
    nbElements=len(elements[1:])
#    if not nbElements in nbIndiceNotes: 
#        print "Ajouter une répartition des notes pour %d indices"%nbElements
#        print titre
#        print elements[1:]
    for n,element in enumerate(elements[1:]):
        (reponseSpec,feedbackSpec)=element.split("#")
        reponseSpec=modifierCorrection(reponseSpec)
        if feedbackSpec:
            for mot in motReferences:
                if mot in feedbackSpec:
                    feedbackSpec=lierMot(mot,feedbackSpec)
        corrections.append((nbIndiceNotes(nbElements,percent=0)[n],reponseSpec,feedbackSpec))
#    print nbMots
    minMots=("0","^"+"([^ ]+ +)?"*(nbMots-4)+"[^ ]+$",msgPlusDeMots)
    maxMots=("0","^"+"([^ ]+ +)"*(nbMots+2)+"([^ ]+ +)*[^ ]+$",msgMoinsDeMots)
#    print "~".join(elements)
    result.extend([solution,minMots,maxMots])
    result.extend(corrections)
    '''
    for note,sol,fb in result:
        print "note",note
        print "sol",sol 
        print "fb",fb
    '''
    return result

In [28]:
consigne="<br/>".join(configuration[u"Consigne"])
motReferences=configuration[u"Références"]
separationExercices=[]
for question in donnees:
    for nom in question:
        titre=nom
        consigneLignes=[consigne]
        consigneLignes.append(u"".join(re.split("[ -]",question[nom]["boucle"][0])))
#        questionLignes.append(u"".join(re.split("[ -]",question[nom]["boucle"][0])))
#        questionLignes.append(u"".join(question[nom]["boucle"][0].split(" ")))
        questionLignes=separerMots(question[nom]["boucle"],titre)
#        print questionLignes
#        textQuestion=u"<br/>".join(questionLignes)
        separationExercices.append(RegExExercice(titre,consigneLignes,questionLignes,penalty=adaptatifPenalite))

ToolsMoodleXML2.py:290: UserWarning: 
Plusieurs réponses dans le RegEx
Elles seront toutes considérées correctes.
  warnings.warn("\nPlusieurs réponses dans le RegEx\nElles seront toutes considérées correctes.")


In [29]:
categorie=configuration[u"Catégorie"]
regexQuestions=XMLSeries(categorie)

In [30]:
for exercice in separationExercices:
    regexQuestions.addExercice(exercice)

In [31]:
xmlOutput=MoodleXML()
xmlOutput.addQuiz(regexQuestions.getSeries())

In [32]:
xmlFichier=yamlFichier.split(".")[0]+".xml"

In [33]:
with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
#    print [questions.getClozes()]
    output.write(xmlOutput.getXML())

In [ ]:
motReferences

In [ ]:
correction=u"--.*(&& gris(M)[SG.⊖] && vert(M)[SG.⊖] && clair[M.SG.⊖]).*"
#correction=u"--.*(&& DEF-M.SG.⊖ chat&& DEF-F.SG.⊖ gouttière).*"
(prefixe, corps, suffixe)=re.match(ur"^(--\.\*\()(&&.*)(\)\.\*)$",correction).groups()
print corps
subParentheses = re.compile(ur"\(([^)]*)\)")
result=subParentheses.sub(ur"\\(\1\\)",corps)
print result
subCrochets = re.compile(ur"\[([^]]*)\]")
result=subCrochets.sub(ur"\\[\1\\]",result)
subPoints = re.compile(ur"(\.[^*])")
result=subPoints.sub(ur"\\\1",result)

print prefixe+result+suffixe

In [ ]:
subParentheses = re.compile(ur"&&([^&)]*)\(([^)]*)\)")
subCrochets = re.compile(ur"&&([^&[]*)\[([^\]]*)\]")
subPoints = re.compile(ur"&&(([^)\.]|\\\)|\\\.)+)(\.[^*])")
print subPoints.search(correction)
result=subParentheses.sub(ur"&&\1\\(\2\\)",correction)
print result
result=subCrochets.sub(ur"&&\1\\[\2\\]",result)
print result
result=subPoints.sub(ur"&&\1\\\3",result)
print result